In [1]:
from __future__ import absolute_import,division,print_function

import tensorflow as tf
import numpy as np
import os
import random
from PIL import Image as img
import cv2
from keras.optimizers import Adam

from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, Flatten, Activation
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing import image

from keras.models import load_model

Using TensorFlow backend.


In [2]:
def abs_diff_output_shape(input_shapes):
  shape1, shape2, shape3= input_shapes
  return shape1


def distance_vec(y_pred): #Cosine distance
  x = y_pred[0]
  y = y_pred[1]
  z = y_pred[2]
  negative = K.prod(K.stack([x, y], axis=1), axis=1)
  return negative
  
  
  anchor = y_pred[0]
  positive = y_pred[1]
  negative = y_pred[2]

def get_abs_diff(y_pred):
    # L1 distance between two vectors
    x = y_pred[0]
    y = y_pred[1]
    z = y_pred[2]
    return K.abs(x - y)


def triple_loss(y_true, y_pred):   # y_pred contiene la imagen ancla, positiva y negativa
  alpha = 0.1
  
  anchor = y_pred[0]
  positive = y_pred[1]
  negative = y_pred[2]
    
  positive_distance = K.mean(K.square(anchor-positive),axis=-1)  #En principio mean y sum hacen lo mismo excepto que una hace media y otro suma
  negative_distance = K.mean(K.square(anchor-negative),axis=-1)
  
  loss = positive_distance - negative_distance
  return K.maximum(loss + alpha, 0)   

In [3]:
def get_random_image(img_groups, group_names, gid):
    gname = group_names[gid]
    photos = img_groups[gname]
    pid = np.random.choice(np.arange(len(photos)), size=1)[0]
    pname = photos[pid]
    return gname + pname + ".jpg"
   
def create_triplet(number_images,minImage,maxImage): # Crea tripleta
  img_groups = {}
  for folder in  sorted(os.listdir('/home/luis/dataset'),reverse=False):
    i = 0 
    for img_file in sorted(os.listdir('/home/luis/dataset/' + folder),reverse=False):
      #if i == number_images: # Para solo coger 40 fotos
        #break
      if i < minImage:
        i = i + 1
        continue
      if i > maxImage:
        break
      prefix, suffix = img_file.split(".")
      pid = prefix[3:]
      if folder in img_groups:
          img_groups[folder].append(pid)
      else:
          img_groups[folder] = [pid]
      i += 1
  pos_triples, neg_triples, labels = [], [], []
  # positive pairs and negative 
  group_names = list(img_groups.keys())
  for key in img_groups.keys():
    for i in range(0,number_images):
      for j in range(i+1,number_images): # Si dejo i me cogeria duplas con elementos iguales osea (001,001)
        inc = random.randrange(1, classes+1)
        dn = (int(key) + inc) % classes
        right = get_random_image(img_groups, group_names, dn)
        pos_triples.append((key + img_groups[key][i] + ".jpg", key + img_groups[key][j] + ".jpg", right))
  return pos_triples  

def load_image(image_name):
    image = cv2.imread('/home/luis/dataset/' + image_name[0:3] + '/' + image_name)
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    return image     
           
def triples_batch(tamaño,image_triples):#, batch_size): #Tripletas
    images = np.empty([tamaño,3,96,96,3])
    for index,i in enumerate(image_triples):
        ahs, phs, nhs= i
        a = (load_image(ahs),load_image(phs),load_image(nhs))
        images[index] = a
    return images

In [22]:
def get_random_image(img_groups, group_names, gid):     #Este metodo es para juntar todas con todas
    gname = group_names[gid]
    photos = img_groups[gname]
    pid = np.random.choice(np.arange(len(photos)), size=1)[0]
    pname = photos[pid]
    return gname + pname + ".jpg"
   
def create_triplet(number_images,minImage,maxImage): # Crea tripleta
  img_groups = {}
  for folder in  sorted(os.listdir('/home/luis/dataset'),reverse=False):
    i = 0 
    for img_file in sorted(os.listdir('/home/luis/dataset/' + folder),reverse=False):
      #if i == number_images: # Para solo coger 40 fotos
        #break
      if i < minImage:
        i = i + 1
        continue
      if i > maxImage:
        break
      print(img_file)
      prefix, suffix = img_file.split(".")
      pid = prefix[3:]
      if folder in img_groups:
          img_groups[folder].append(pid)
      else:
          img_groups[folder] = [pid]
      i += 1
  pos_triples = []
  # positive pairs and negative 
  group_names = list(img_groups.keys())
  for key in img_groups.keys():
    for i in range(0,number_images):
        for j in range(i+1,number_images): # Si dejo i me cogeria duplas con elementos iguales osea (001,001)
            for k in img_groups.keys():
                if k != key:
                    for l in range(0,number_images):
                        print((key + img_groups[key][i] + ".jpg", key + img_groups[key][j] + ".jpg", k + img_groups[k][l]))
                        pos_triples.append((key + img_groups[key][i] + ".jpg", key + img_groups[key][j] + ".jpg", k + img_groups[k][l]))
  return pos_triples

def load_image(image_name):
    image = cv2.imread('/home/luis/dataset/' + image_name[0:3] + '/' + image_name)
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    return image     
           
def triples_batch(tamaño,image_triples):#, batch_size): #Tripletas
    images = np.empty([tamaño,3,96,96,3])
    for index,i in enumerate(image_triples):
        ahs, phs, nhs= i
        a = (load_image(ahs),load_image(phs),load_image(nhs))
        images[index] = a
    return images

In [4]:
def create_base_network(input_dim, classes): # SmallerVGGnet
  red = Sequential() 
  red.add(Conv2D(32, (3, 3), padding="same",input_shape=input_dim))
  red.add(Activation("relu"))
  red.add(BatchNormalization(axis=-1))
  red.add(MaxPooling2D(pool_size=(3, 3)))
  #red.add(Dropout(0.25))
  
  # (CONV => RELU) * 2 => POOL
  red.add(Conv2D(64, (3, 3), padding="same"))
  red.add(Activation("relu"))
  red.add(BatchNormalization(axis=-1))
  red.add(Conv2D(64, (3, 3), padding="same"))
  red.add(Activation("relu"))
  red.add(BatchNormalization(axis=-1))
  red.add(MaxPooling2D(pool_size=(2, 2)))
  #red.add(Dropout(0.25))
  
  # (CONV => RELU) * 2 => POOL
  red.add(Conv2D(128, (3, 3), padding="same"))
  red.add(Activation("relu"))
  red.add(BatchNormalization(axis=-1))
  red.add(Conv2D(128, (3, 3), padding="same"))
  red.add(Activation("relu"))
  red.add(BatchNormalization(axis=-1))
  red.add(MaxPooling2D(pool_size=(2, 2)))
  #red.add(Dropout(0.25))
  
	# first (and only) set of FC => RELU layers
  red.add(Flatten())
  #red.add(Dense(2048))
  #red.add(Activation("relu"))
  #red.add(BatchNormalization())
  #red.add(Dropout(0.5))

  red.add(Dense(1024))
  red.add(Activation("relu"))
  red.add(BatchNormalization())
  red.add(Dropout(0.5))

  return red 

In [5]:
number_images = 4 # Numero de imagenes cogidas
classes = 151 # Numero de pokemones
batch = 32
nb_epoch = 10

input_dim = (96,96,3) # Son el ancho el alto y la profundidad de la imagen, 3 al ser en color es la variable input_dim pero todavia no esta tocada

In [6]:
triples_data = create_triplet(number_images,0,3)

print("# image triples:", len(triples_data))

tm_train = 800
tm_val = 906

tr_train = triples_data[0:tm_train]
tr_val = triples_data[tm_train:tm_val]  #El tamaño tiene que ser par (No se mucho por que)

print("# image triples:", len(tr_train))
print("# image triples:", len(tr_val))

image_cache = {}

tr_load_train = triples_batch(800,tr_train)
tr_load_val = triples_batch(106,tr_val)



# image triples: 906
# image triples: 800
# image triples: 106


In [7]:
print("# image train triples:", len(tr_train))
[x for x in tr_train[0:5]]

print("# image validation triples:", len(tr_val))
[x for x in tr_val[0:5]]

print(tr_load_train.shape)

print(tr_load_val.shape)

# image train triples: 800
# image validation triples: 106
(800, 3, 96, 96, 3)
(106, 3, 96, 96, 3)


In [8]:
# network definition
base_network = create_base_network(input_dim, classes)
#base_network.summary()


anchor_a = Input(shape=(input_dim))   #Nos da un tensor del tamaño input_dim
anchor_p = Input(shape=(input_dim))
anchor_n = Input(shape=(input_dim))


processed_a = base_network(anchor_a)
processed_p = base_network(anchor_p)
processed_n = base_network(anchor_n)

distance = Lambda(distance_vec, output_shape=abs_diff_output_shape)([processed_a, processed_p, processed_n])

pred = Dense(1, activation = 'sigmoid')(distance)

model = Model(inputs=[anchor_a, anchor_p, anchor_n], outputs=pred)


In [9]:
def reshape(data,dim):
  data_a = data[:, 0].reshape(-1, dim[0], dim[1], dim[2])
  data_p = data[:, 1].reshape(-1, dim[0], dim[1], dim[2])
  data_n = data[:, 2].reshape(-1, dim[0], dim[1], dim[2])
  return [data_a,data_p,data_n]


In [10]:
#Datos de entrenamiento

tr_load_train_ = reshape(tr_load_train,input_dim)
tr_load_train_a = tr_load_train_[0]
tr_load_train_p = tr_load_train_[1]
tr_load_train_n = tr_load_train_[2]

print(tr_load_train_a.shape)
print(tr_load_train_p.shape)
print(tr_load_train_n.shape)

#Datos de validacion

tr_load_val_ = reshape(tr_load_val,input_dim)
tr_load_val_a = tr_load_val_[0]
tr_load_val_p = tr_load_val_[1]
tr_load_val_n = tr_load_val_[2]

print(tr_load_val_a.shape)
print(tr_load_val_p.shape)
print(tr_load_val_n.shape)

(800, 96, 96, 3)
(800, 96, 96, 3)
(800, 96, 96, 3)
(106, 96, 96, 3)
(106, 96, 96, 3)
(106, 96, 96, 3)


In [11]:
INIT_LR = 1e-3
EPOCHS = 100
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss=triple_loss, optimizer=opt, metrics=["accuracy"])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1024)         8673152     input_1[0][0]                    
                                                                 input_2[0][0]                    
          

In [12]:
train = np.ones(len(tr_load_train))
val = np.ones(len(tr_load_val))
a = model.fit([tr_load_train_a,tr_load_train_a,tr_load_train_a], train, batch_size=batch, epochs=nb_epoch, validation_data=([tr_load_val_a,tr_load_val_p,tr_load_val_n], val))

Train on 800 samples, validate on 106 samples
Epoch 1/10
800/800 [==============================] - 45s 56ms/step - loss: 0.1605 - acc: 0.5525 - val_loss: 0.3924 - val_acc: 0.2547
Epoch 2/10
768/800 [===========================>..] - ETA: 1s - loss: 0.1753 - acc: 0.6589

KeyboardInterrupt: 

In [ ]:
for i in range(0,(int)(40/number_images)):
    triples_data = create_triplet(number_images,i*number_images,i*number_images+number_images-1)

    print("# image triples:", len(triples_data))

    tm_train = 800
    tm_val = 906

    tr_train = triples_data[0:tm_train]
    tr_val = triples_data[tm_train:tm_val]  #El tamaño tiene que ser par (No se mucho por que)

    print("# image triples:", len(tr_train))
    print("# image triples:", len(tr_val))

    tr_load_train = triples_batch(800,tr_train)
    tr_load_val = triples_batch(106,tr_val)

    #Datos de entrenamiento

    tr_load_train_ = reshape(tr_load_train,input_dim)
    tr_load_train_a = tr_load_train_[0]
    tr_load_train_p = tr_load_train_[1]
    tr_load_train_n = tr_load_train_[2]

    #Datos de validacion

    tr_load_val_ = reshape(tr_load_val,input_dim)
    tr_load_val_a = tr_load_val_[0]
    tr_load_val_p = tr_load_val_[1]
    tr_load_val_n = tr_load_val_[2]

    train = np.ones(len(tr_load_train))
    val = np.ones(len(tr_load_val))
    a = model.fit([tr_load_train_a,tr_load_train_a,tr_load_train_a], train, batch_size=batch, epochs=nb_epoch, validation_data=([tr_load_val_a,tr_load_val_p,tr_load_val_n], val))

In [ ]:
model.save('my_model.h5')